### Data analysis notebook for Hall Effect experiment
- Loads in voltage data taken with using the Van de Pauw method for determining resistivity
- purpose is to experiment with analysis techniques eg. fitting methods and error propagation
- not intended to produce final results or plots

In [91]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [92]:
path = '/Users/Bryn_Lloyd/Documents/Uni/Year 3/Labs/Hall Effect/'

In [93]:
vdp_data = pd.read_csv(f'{path}van_der_pauw_correction_factor.csv', delimiter=',')
data = pd.read_csv(f'{path}n-type GaAs resistivity_data.csv', delimiter=',')

In [94]:
resistance = []
R_error = []
for i in range(len(data)):
    resistance_value = data['Voltage'][i] / data['Current'][i]
    resistance_error = resistance_value * np.sqrt((data['V_error'][i]/data['Voltage'][i])**2 + (data['C_error'][i]/data['Current'][i])**2)
    resistance.append(resistance_value)
    R_error.append(resistance_error)
    

data['Resistance'] = resistance
data['R_error'] = R_error

data

,Current,C_error,Voltage,V_error,Resistance,R_error
0,2.0010,0.0020,14.419,0.003,7.205897,0.007357
1,2.0010,0.0020,14.421,0.003,7.206897,0.007358
2,2.0010,0.0020,14.415,0.003,7.203898,0.007355
3,2.0010,0.0020,14.416,0.003,7.204398,0.007355
4,2.0010,0.0020,4.259,0.003,2.128436,0.002603
5,2.0010,0.0020,4.258,0.003,2.127936,0.002602
6,2.0010,0.0020,4.257,0.003,2.127436,0.002602
7,2.0010,0.0020,4.257,0.003,2.127436,0.002602
8,3.0010,0.0030,21.623,0.003,7.205265,0.007272
9,3.0010,0.0030,21.623,0.003,7.205265,0.007272


In [95]:
N_MEASUREMENTS = 3

R_AB_list = []
R_AB_error_list = []

for j in range(2*N_MEASUREMENTS):
    R_AB = 0
    temp_error_list=[]
    for k in range(4):
        R_AB += data['Resistance'][k+ 4*j]
        temp_error_list.append((data['R_error'][k+4*j])**2)
    R_AB_list.append(R_AB/4)
    R_AB_error_list.append(np.sqrt(np.sum(temp_error_list)))


R_A = R_AB_list[::2]
R_A_error = R_AB_error_list[::2]
R_B = R_AB_list[1::2]
R_B_error = R_AB_error_list[1::2]

print(f'R_A = {R_A} +/- {R_A_error} ohms')
print(f'R_B = {R_B} +/- {R_B_error} ohms')


R_A = [7.205272363818091, 7.206014661779407, 7.208383353341337] +/- [0.014712148164478916, 0.014545292170904186, 0.014300928095319142] ohms
R_B = [2.127811094452774, 2.128457180939687, 2.130852340936374] +/- [0.005204155157161509, 0.004701763526922441, 0.007124038022044338] ohms


In [96]:
vdp_fit = np.polyfit(vdp_data['x'],vdp_data['y'],20)
vdp_pfit = np.poly1d(vdp_fit)

#plt.plot(vdp_data['x'],vdp_data['y'],'x')
#plt.plot(vdp_data['x'],vdp_pfit(vdp_data['x']))


/Users/Bryn_Lloyd/Library/Python/3.11/lib/python/site-packages/IPython/core/interactiveshell.py:3433: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)


In [97]:
R_sh = []
R_sh_error = []

for i in range(N_MEASUREMENTS):
    f = vdp_pfit(R_A[i]/R_B[i])
    R_sh.append((np.pi * (R_A[i]+R_B[i]))/(2*np.log(2)) * f)
    R_sh_error.append(np.sqrt((R_A_error[i])**2 + (R_B_error[i])**2))

print(f'R_sh = {R_sh} +/- {R_sh_error} ohms?')

R_sh = [18.88548701868719, 18.888880030791604, 18.900836570537006] +/- [0.015605464892575006, 0.015286337187176815, 0.01597712308668309] ohms?


In [100]:
THICKNESS = 3e-6

resistivity  = []
for i in range(N_MEASUREMENTS):
    resistivity.append(R_sh[i] * THICKNESS)

rho = np.sum(resistivity)/N_MEASUREMENTS
R_sh_error = np.array(R_sh_error)
rho_relative_error = np.sqrt((np.sum(R_sh_error**2))/(np.sum(R_sh)/3))
rho_error = rho*rho_relative_error

print(f'Resistivity = {rho} +/- {rho_error} ohm meters')

Resistivity = 5.66752036200158e-05 +/- 3.529006183414019e-07 ohm meters
